# Predictions

## Data wrangling

In [1]:
# Preamble
import pandas as pd
import numpy as np
pd.set_option("mode.chained_assignment", None)
import random
random.seed(1509)
import matplotlib.pyplot as plt
import lightgbm as lgb
import pyarrow.feather as feather
from os import chdir, getcwd
import statsmodels.api as sm
from pprint import pprint
from nested_cv import NestedCV

# sci-kit
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn import tree

In [2]:
data_dir = '/home/jovyan/work/Data/'
results_dir = '/home/jovyan/work/Results/'

In [3]:
# select_features = ['reporter.ISO', 'partner.ISO', 'year',
#                    'ln.Tot_IFF_t', 'ln.In_Tot_IFF_t',
#                    'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
#                    'dist', 'contig', 
#                    'comlang', 'comcol', 'col45', 
#                    'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
#                    'rCorrCont', 'pCorrCont',
#                    'rRegQual', 'pRegQual', 
#                    'rRuleLaw', 'pRuleLaw',
#                    'rSecrecyScore', 'pSecrecyScore',
#                    'rFSI.rank', 'pFSI.rank',
#                    'rKFSI13', 'pKFSI13',
#                    'rKFSI17', 'pKFSI17',
#                    'rKFSI20', 'pKFSI20',
#                    'rFATF', 'pFATF',
#                    'ihs.tariff',
#                    'kai_o', 'kai_d', 'kao_o', 'kao_d',
#                    'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
#                    'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']
select_features = ['reporter.ISO', 'partner.ISO', 'year',
                   'ln.Tot_IFF_t', 'ln.In_Tot_IFF_t',
                   'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
                   'dist', 'contig', 
                   'comlang', 'comcol', 'col45', 
                   'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
                   'rCorrCont', 'pCorrCont',
                   'rRegQual', 'pRegQual', 
                   'rRuleLaw', 'pRuleLaw',
                   'pSecrecyScore',
                   'pFSI.rank',
                   'pKFSI13',
                   'pKFSI17',
                   'pKFSI20',
                   'rFATF', 'pFATF',
                   'ihs.tariff',
                   'kai_o', 'kai_d', 'kao_o', 'kao_d',
                   'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
                   'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']

features = [       'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
                   'dist', 'contig', 
                   'comlang', 'comcol', 'col45', 
                   'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
                   'rCorrCont', 'pCorrCont',
                   'rRegQual', 'pRegQual', 
                   'rRuleLaw', 'pRuleLaw',
                   'pSecrecyScore',
                   'pFSI.rank',
                   'pKFSI13',
                   'pKFSI17',
                   'pKFSI20',
                   'rFATF', 'pFATF',
                   'ihs.tariff',
                   'kai_o', 'kai_d', 'kao_o', 'kao_d',
                   'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
                   'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']

ids = [       'reporter.ISO', 'partner.ISO', 'year']

## Subset sample

In [72]:
data = feather.read_feather(results_dir + 'Africa.feather')

In [5]:
def create_smp(data, features):
    """
    Create train and test samples that are complete.
    """
    smp = data[features]
    smp.dropna(axis=0, how='any', inplace=True)
    return smp

In [73]:
data_smp = create_smp(data, select_features)

In [74]:
idx = data_smp[ids]
X = data_smp[features]
Y_out = data_smp[['ln.Tot_IFF_t']]
Y_in = data_smp[['ln.In_Tot_IFF_t']]

In [75]:
feather.write_feather(idx, results_dir + 'idx.feather')
feather.write_feather(X, results_dir + 'X.feather')
feather.write_feather(Y_out, results_dir + 'Y_out.feather')
feather.write_feather(Y_in, results_dir + 'Y_in.feather')

In [76]:
print('X: ', X.shape)
print('Y_out: ', Y_out.shape)
print('idx: ', idx.shape)

X:  (54662, 42)
Y_out:  (54662, 1)
idx:  (54662, 3)


### Train/test split

In [ ]:
train_agg_smp = create_smp(train_agg, select_features)
test_agg_smp = create_smp(test_agg, select_features)

In [ ]:
feather.write_feather(train_agg_smp, results_dir + 'train_agg_smp.feather')
feather.write_feather(test_agg_smp, results_dir + 'test_agg_smp.feather')

In [ ]:
Y_train_out = train_agg_smp[['ln.Tot_IFF_t']]
Y_train_in = train_agg_smp[['ln.In_Tot_IFF_t']]
X_train = train_agg_smp[features]
Y_test_out = test_agg_smp[['ln.Tot_IFF_t']]
Y_test_in = test_agg_smp[['ln.In_Tot_IFF_t']]
X_test = test_agg_smp[features]

In [ ]:
print('X_train: ', X_train.shape, '\nX_test: ',  X_test.shape)

## Linear regression

### Fit linear regression model

In [39]:
linear_mod_out = LinearRegression()  
linear_mod_out.fit(X.values, Y_out.values)

LinearRegression()

In [ ]:
linear_mod_in = LinearRegression()  
linear_mod_in.fit(X_train.values, Y_train_in.values)

### Print coefficients

In [ ]:
print(linear_mod_out.intercept_)
print(linear_mod_out.coef_)

In [ ]:
print(linear_mod_in.intercept_)
print(linear_mod_in.coef_)

In [ ]:
# Xconst = sm.add_constant(X_train)
# est = sm.OLS(Y_train_out, Xconst)
# est2 = est.fit()
# print(est2.summary())

In [ ]:
# Xconst = sm.add_constant(X_train)
# est = sm.OLS(Y_train_in, Xconst)
# est2 = est.fit()
# print(est2.summary())

### Predictions

In [ ]:
preds_LM_train_out = linear_mod_out.predict(X_train)
preds_LM_test_out = linear_mod_out.predict(X_test)

In [ ]:
preds_LM_train_in = linear_mod_in.predict(X_train)
preds_LM_test_in = linear_mod_in.predict(X_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_LM_train_out), results_dir + 'preds.LM.train_out_agg.feather')
feather.write_feather(pd.DataFrame(preds_LM_test_out), results_dir + 'preds.LM.test_out_agg.feather')

In [ ]:
feather.write_feather(pd.DataFrame(preds_LM_train_in), results_dir + 'preds.LM.train_in_agg.feather')
feather.write_feather(pd.DataFrame(preds_LM_test_in), results_dir + 'preds.LM.test_in_agg.feather')

### Predictive accuracy

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_LM_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_LM_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_LM_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_LM_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_LM_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_LM_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_LM_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_LM_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_LM_test_out))  
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_LM_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_LM_test_out))
print('Dollar RMSE of the training set (outflows):', np.mean(np.square((np.exp(Y_train_out.values)*10**3 - np.exp(preds_LM_train_out)*10**3))) / 10**9)
print('Dollar RMSE of the test set (outflows):', np.mean(np.square((np.exp(Y_test_out.values)*10**3 - np.exp(preds_LM_test_out)*10**3))) / 10**6)

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_LM_test_in))
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_LM_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_LM_test_in))
print('Dollar RMSE of the test set (inflows):', np.mean(np.square((np.exp(Y_train_in.values)*10**3 - np.exp(preds_LM_train_in)*10**3))) / 10**9)
print('Dollar RMSE of the test set (inflows):', np.mean(np.square((np.exp(Y_test_in.values)*10**3 - np.exp(preds_LM_test_in)*10**3))) / 10**9)

### Cross-validation

In [ ]:
linear_mod_out = LinearRegression()  
linear_mod_out.fit(X.values, Y_out.values)

In [45]:
scores = cross_val_score(linear_mod_out, X, Y_out.values.ravel(), cv = 10)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.45186556  0.4127257   0.52329192  0.55024428  0.51445511  0.51008895
  0.41924502  0.50052528  0.48595955 -0.0046091 ]


In [46]:
scores.mean()

0.43637922845408933

In [ ]:
predictions = cross_val_predict(linear_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [ ]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.LM.CV_out.feather')

## Random Forests

### Fit baseline random forests regression

In [10]:
RF_0_mod_out = RandomForestRegressor(random_state = 1509)
RF_0_mod_out.fit(X, Y_out.values.ravel())

In [11]:
RF_0_mod_in = RandomForestRegressor(random_state = 1509)
RF_0_mod_in.fit(X, Y_in.values.ravel())

### Tune hyperparameters with randomized search

In [12]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 3000, num = 100)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 500, num = 100)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 10)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(1, 100, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'bootstrap': bootstrap}
print(random_grid)

In [13]:
# Create the base model to tune
RF_0_mod_out = RandomForestRegressor(random_state = 1509)

# Random search of parameters on base model using 3 fold cross validation 
# Search across 100 different combinations, and use all available cores
RF_random_out = RandomizedSearchCV(random_state = 1509,
                                   estimator = RF_0_mod_out, 
                                   param_distributions = random_grid,
                                   scoring = 'r2',
                                   n_iter = 100, cv = 3, 
                                   verbose = 3, n_jobs = -1)

In [14]:
# Fit the random search model
RF_random_out.fit(X, Y_out.values.ravel())

In [24]:
print('Best Score: %s' % RF_random_out.best_score_)

Best Score: 0.42809867095426846


In [26]:
RF_random_out.best_params_

{'n_estimators': 1278,
 'min_samples_split': 23,
 'min_samples_leaf': 23,
 'max_features': 'auto',
 'max_depth': 125,
 'bootstrap': True}

In [ ]:
RF_random_out.cv_results_

In [32]:
RF_random_out.best_score_

0.42809867095426846

In [59]:
best_random = RF_random_out.best_estimator_
print(best_random.score(X, Y_out))

0.6225641819292647


In [33]:
best_random

0.7514533672429826

In [60]:
cross_val_score(RF_random_out.best_estimator_, X, Y_out.values.ravel(), cv = 3)

array([ 0.17849742, -0.12171697,  0.22869577])

In [30]:
best_random.score(X, Y_out)

0.7514533672429826

In [20]:
scores = cross_val_score(best_random, X, Y_out.values.ravel(), cv = 6)
print('Cross-validated scores:', scores)

Cross-validated scores: [0.47272326 0.48149061 0.33351371 0.43778015 0.52023699 0.24338979]


In [21]:
scores.mean()

0.41485575057312524

In [22]:
predictions = cross_val_predict(best_random, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

0.494455408385449

In [ ]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.RF.CV_out.feather')

### Tune hyperparameters with grid search

In [47]:
# Create the base model to tune (with some parameters from random search)
RF_base_mod_out  = RandomForestRegressor(random_state = 1509,
                                         max_features = 'auto',
                                         bootstrap = True)

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [500, 2500],
    'max_depth': [10, 300],
    'min_samples_split': [8, 80],
    'min_samples_leaf': [8, 80],
}

# Grid search of parameters on base model using 3 fold cross validation 
# Search all possible cominations, and use all available cores
RF_grid_out = GridSearchCV(estimator = RF_base_mod_out, 
                           param_grid = param_grid,
                           scoring = 'r2',
                           verbose = 3, n_jobs = -1)

In [48]:
# Fit the grid search model
RF_grid_out.fit(X, Y_out.values.ravel())

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=RandomForestRegressor(random_state=1509), n_jobs=-1,
             param_grid={'max_depth': [10, 300], 'min_samples_leaf': [8, 80],
                         'min_samples_split': [8, 80],
                         'n_estimators': [500, 2500]},
             scoring='r2', verbose=3)

[CV 2/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=500;, score=0.296 total time=  40.2s
[CV 4/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=500;, score=0.381 total time=  39.9s
[CV 1/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.481 total time= 3.3min
[CV 3/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.552 total time= 3.3min
[CV 5/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.448 total time= 3.3min
[CV 1/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.490 total time= 2.8min
[CV 3/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.543 total time= 2.8min
[CV 5/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.417 total time= 2.8min
[CV 2/5] END max_depth=10, min_samples_leaf=80,

In [49]:
print('Best Score: %s' % RF_grid_out.best_score_)

Best Score: 0.4384566873340452


In [50]:
RF_grid_out.best_params_

{'max_depth': 300,
 'min_samples_leaf': 8,
 'min_samples_split': 80,
 'n_estimators': 2500}

In [65]:
best_grid = RF_grid_out.best_estimator_
print(best_grid.score(X, Y_out))

0.6113986943864123


In [66]:
best_grid.fit(X, Y_out.values.ravel())

RandomForestRegressor(max_depth=300, min_samples_leaf=8, min_samples_split=80,
                      n_estimators=2500, random_state=1509)

In [67]:
preds_RF_out = best_grid.predict(X)
r2_score(Y_out, preds_RF_out)

0.7057793564482469

In [52]:
scores = cross_val_score(best_grid, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [0.37440636 0.3426772  0.48558579]


In [53]:
scores.mean()

0.4008897832777349

### Manually configure hyperparameters

In [77]:
RF_mod_out = RandomForestRegressor(random_state = 1509,
                                   n_estimators = 1278,
                                   max_depth = 125,
                                   min_samples_split = 23,
                                   min_samples_leaf = 23,
                                   max_features = 'auto',
                                   bootstrap = True)

In [78]:
# Fit the manually configured model
RF_mod_out.fit(X, Y_out.values.ravel())

RandomForestRegressor(max_depth=125, min_samples_leaf=23, min_samples_split=23,
                      n_estimators=1278, random_state=1509)

In [80]:
print(RF_mod_out.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 125, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 23, 'min_samples_split': 23, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1278, 'n_jobs': None, 'oob_score': False, 'random_state': 1509, 'verbose': 0, 'warm_start': False}


In [81]:
preds_RF_out = RF_mod_out.predict(X)
r2_score(Y_out, preds_RF_out)

0.34594088525523703

In [ ]:
predictions = cross_val_predict(RF_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [64]:
scores = cross_val_score(RF_mod_out, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.17849742 -0.12171697  0.22869577]


### Predictions

In [ ]:
# Base model
# preds_RF_train_out = RF_0.predict(X_train)
# preds_RF_test_out = RF_0.predict(X_test)

In [ ]:
# Manually configured model (outflows)
preds_RF_train_out = RF_mod_out.predict(X_train)
preds_RF_test_out = RF_mod_out.predict(X_test)

In [ ]:
# Model tuned with randomized and grid search (outflows)
preds_RF_train_out = RF_grid_out.predict(X_train)
preds_RF_test_out = RF_grid_out.predict(X_test)

In [ ]:
# Manually configured model (inflows)
preds_RF_train_in = RF_mod_in.predict(X_train)
preds_RF_test_in = RF_mod_in.predict(X_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_RF_train_out), results_dir + 'preds.RF.train_out_agg.feather')
feather.write_feather(pd.DataFrame(preds_RF_test_out), results_dir + 'preds.RF.test_out_agg.feather')

In [ ]:
feather.write_feather(pd.DataFrame(preds_RF_train_in), results_dir + 'preds.RF.train_in_agg.feather')
feather.write_feather(pd.DataFrame(preds_RF_test_in), results_dir + 'preds.RF.test_in_agg.feather')

### Predictive accuracy

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_RF_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_RF_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_RF_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_RF_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_RF_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_RF_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_RF_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_RF_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_RF_test_out))
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_RF_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_RF_test_out))
# print('Dollar RMSE of the training set (outflows):', np.mean(np.square((np.exp(Y_train_out.values) - np.exp(preds_RF_train_out)))))
# print('Dollar RMSE of the test set (outflows):', np.mean(np.square((np.exp(Y_test_out.values) - np.exp(preds_RF_test_out)))))

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_RF_test_in))  
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_RF_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_RF_test_in))
#print('Dollar RMSE of the training set (inflows):', np.mean(np.square(np.exp(Y_train_in.values)/10**3 - np.exp(preds_RF_train_in)/10**3)))
#print('Dollar RMSE of the test set (inflows):', np.mean(np.square(np.exp(Y_test_in.values)/10**3 - np.exp(preds_RF_test_in)/10**3)))

### Cross-validation

In [ ]:
RF_mod_CV = RandomForestRegressor(random_state = 1509,
                                  n_estimators = 1278,
                                  max_depth = 125,
                                  min_samples_split = 23,
                                  min_samples_leaf = 23,
                                  max_features = 'auto',
                                  bootstrap = True)

In [ ]:
scores = cross_val_score(RF_mod_CV, X, Y_out.values.ravel(), cv = 6)
print('Cross-validated scores:', scores)

In [ ]:
scores.mean()

In [68]:
predictions = cross_val_predict(best_grid, X, Y_out.values.ravel(), cv = 3)
r2_score(Y_out, predictions)

0.21085539345854087

In [69]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.RF.CV_out.feather')

### Visualize results

In [ ]:
feat_importances = pd.Series(best_random.feature_importances_, index = X.columns)
indices = np.argsort(-1*feat_importances)
names = [X_train.columns[i] for i in indices]
fig, ax = plt.subplots(figsize = (10, 10))
plt.barh(range(X_train.shape[1]), feat_importances[indices], color = 'red')
plt.yticks(range(X_train.shape[1]), names, fontsize = 12)
plt.gca().invert_yaxis()
ax.set_xlabel('Feature importance')
plt.title("Random Forest - Feature importance for illicit outflows", fontsize = 16)
plt.savefig(results_dir + 'RF_feature_importance_out.png')

In [ ]:
feat_importances = pd.Series(RF_mod_in.feature_importances_, index=X_train.columns)
indices = np.argsort(-1*feat_importances)
names = [X_train.columns[i] for i in indices]
fig, ax = plt.subplots(figsize = (10, 10))
plt.barh(range(X_train.shape[1]), feat_importances[indices], color = 'red')
plt.yticks(range(X_train.shape[1]), names, fontsize = 12)
plt.gca().invert_yaxis()
ax.set_xlabel('Feature importance')
plt.title("Random Forest - Feature importance for illicit inflows", fontsize = 16)
plt.savefig(results_dir + 'RF_feature_importance_in.png')

In [ ]:
tree.export_graphviz(RF_mod_out.estimators_[0])

In [ ]:
tree.plot_tree(RF_mod_out.estimators_[0])

### Nested cross-validation

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 3000, num = 100)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 500, num = 100)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 10)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(1, 100, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the parameter grid
param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'max_features': max_features,
              'bootstrap': bootstrap}

In [ ]:
# Perform nested cross-validation
RF_mod_NCV = NestedCV(model = RandomForestRegressor(random_state = 1509), 
                      params_grid = param_grid,
                      outer_kfolds = 5, inner_kfolds = 5,
                      n_jobs = -1,
                      cv_options={'metric': r2_score,
                                  'randomized_search': True,
                                  'randomized_search_iter': 10})

In [ ]:
# Fit the nested cross-validated model
RF_mod_NCV.fit(X = X, y = Y_out.values.ravel())

In [ ]:
RF_mod_NCV.outer_scores

In [ ]:
np.mean(RF_mod_NCV.outer_scores)

In [ ]:
RF_mod_NCV.best_params

## Light GBM

In [ ]:
lightGBM_train_out = lgb.Dataset(X_train, Y_train_out)
lightGBM_test_out = lgb.Dataset(X_test, Y_test_out)
lightGBM_train_in = lgb.Dataset(X_train, Y_train_in)
lightGBM_test_in = lgb.Dataset(X_test, Y_test_in)

In [ ]:
params = {
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 100,
    'max_depth': 10,
    'learning_rate' : 0.1,
    'feature_fraction' : 0.6,
    'verbosity' : -1,
    'random_state' : 1509
}
lightGBM_mod_out = lgb.train(
    params,
    lightGBM_train_out,
    500,
    valid_sets = [lightGBM_train_out, lightGBM_test_out],
    valid_names = ["train", "test"],
    early_stopping_rounds = 50,
    verbose_eval = 500
)

In [ ]:
params = {
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 100,
    'max_depth': 10,
    'learning_rate' : 0.1,
    'feature_fraction' : 0.6,
    'verbosity' : -1,
    'random_state' : 1509
}
lightGBM_mod_in = lgb.train(
    params,
    lightGBM_train_in,
    500,
    valid_sets = [lightGBM_train_in, lightGBM_test_in],
    valid_names = ["train", "test"],
    early_stopping_rounds = 50,
    verbose_eval = 500
)

In [63]:
preds_lightGBM_out = lightGBM_mod_out.predict(X)
r2_score(Y_out, preds_lightGBM_out)

0.722053692562466

In [ ]:
predictions = cross_val_predict(lightGBM_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [64]:
scores = cross_val_score(lightGBM_mod_out, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.17849742 -0.12171697  0.22869577]


In [ ]:
preds_lightGBM_train_out = pd.DataFrame(lightGBM_mod_out.predict(X_train))
preds_lightGBM_test_out = pd.DataFrame(lightGBM_mod_out.predict(X_test))

In [ ]:
preds_lightGBM_train_in = pd.DataFrame(lightGBM_mod_in.predict(X_train))
preds_lightGBM_test_in = pd.DataFrame(lightGBM_mod_in.predict(X_test))

In [ ]:
feather.write_feather(preds_lightGBM_train_out, results_dir + 'preds.lightGBM.train_out_agg.feather')
feather.write_feather(preds_lightGBM_test_out, results_dir + 'preds.lightGBM.test_out_agg.feather')

In [ ]:
feather.write_feather(preds_lightGBM_train_in, results_dir + 'preds.lightGBM.train_in_agg.feather')
feather.write_feather(preds_lightGBM_test_in, results_dir + 'preds.lightGBM.test_in_agg.feather')

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_lightGBM_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_lightGBM_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_lightGBM_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_lightGBM_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_lightGBM_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_lightGBM_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_lightGBM_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_lightGBM_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_lightGBM_test_out))
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_lightGBM_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_lightGBM_test_out))
#print('Dollar RMSE of the training set (outflows):', np.mean(np.square(np.exp(Y_train_out.values)*10**3 - np.exp(preds_lightGBM_train_out)*10**3)) / 10**6)
#print('Dollar RMSE of the test set (outflows):', np.mean(np.square(np.exp(Y_test_out.values)*10**3 - np.exp(preds_lightGBM_test_out)*10**3)) / 10**6)

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_lightGBM_test_in))
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_lightGBM_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_lightGBM_test_in))
#print('Dollar RMSE of the training set (inflows):', np.mean(np.square(np.exp(Y_train_in.values)/10**3 - np.exp(preds_lightGBM_train_in)/10**3)))
#print('Dollar RMSE of the test set (inflows):', np.mean(np.square(np.exp(Y_test_in.values)/10**3 - np.exp(preds_lightGBM_test_in)/10**3)))

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
lgb.plot_importance(lightGBM_mod_out, height = 0.8, ax = ax)
ax.grid(False)
plt.title("LightGBM - Feature importance for illicit outflows", fontsize = 16)
plt.savefig(results_dir + 'LightGBM_feature_importance_out.png')

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
lgb.plot_importance(lightGBM_mod_in, height = 0.8, ax = ax)
ax.grid(False)
plt.title("LightGBM - Feature importance for illicit inflows", fontsize = 16)
plt.savefig(results_dir + 'LightGBM_feature_importance_in.png')

## Support Vector Machine

In [ ]:
SVM_mod = svm.SVR()
SVM_mod.fit(X_train, Y_train.values.ravel())

In [ ]:
preds_SVM_train = SVM_mod.predict(X_train)
preds_SVM_test = SVM_mod.predict(X_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_SVM_train), results_dir + 'preds.SVM.train_agg.feather')
feather.write_feather(pd.DataFrame(preds_SVM_test), results_dir + 'preds.SVM.test_agg.feather')

In [ ]:
print("RMSE of the training set:", np.sqrt(mean_squared_error(Y_train, preds_SVM_train)))
print("R^2 of the training set:", r2_score(Y_train, preds_SVM_train))

In [ ]:
print("RMSE of the test set:", np.sqrt(mean_squared_error(Y_test, preds_SVM_test)))
print("R^2 of the test set:", r2_score(Y_test, preds_SVM_test))

## Neural Networks

In [ ]:
Y_train = train_agg_smp[['Tot_IFF_t']]
X_train = train_agg_smp[features]
Y_test = test_agg_smp[['Tot_IFF_t']]
X_test = test_agg_smp[features]

In [ ]:
std_sc = StandardScaler()
NN_train = std_sc.fit_transform(X_train)
NN_test = std_sc.transform(X_test)

In [ ]:
NN_mod = MLPRegressor(hidden_layer_sizes = (10),
                      activation = "logistic",
                      random_state = 1, 
                      max_iter = 1000).fit(NN_train, Y_train.values.ravel())

In [ ]:
preds_NN_train = NN_mod.predict(NN_train)
preds_NN_test = NN_mod.predict(NN_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_NN_train), results_dir + 'preds.NN.train_agg.feather')
feather.write_feather(pd.DataFrame(preds_NN_test), results_dir + 'preds.NN.test_agg.feather')

In [ ]:
print("RMSE of the training set:", np.sqrt(mean_squared_error(Y_train, preds_NN_train)))
print("R^2 of the training set:", r2_score(Y_train, preds_NN_train))

In [ ]:
print("RMSE of the test set:", np.sqrt(mean_squared_error(Y_test, preds_NN_test)))
print("R^2 of the test set:", r2_score(Y_test, preds_NN_test))

### Tuning hyperparameters

In [ ]:
NN = MLPRegressor(max_iter = 1000)
parameter_space = {
    'hidden_layer_sizes': [(10,20)],
    'activation': ['logistic', 'relu'],
    #'solver': ['sgd', 'adam'],
    #'alpha': [0.0001, 0.05],
    #'learning_rate': ['constant','adaptive'],
}

In [ ]:
NN_mod_tuned = GridSearchCV(NN, parameter_space, n_jobs = -1, cv = 3)

In [ ]:
NN_mod_tuned.fit(NN_train, Y_train.values.ravel())

In [ ]:
print('Best parameters found:\n', NN_mod_tuned.best_params_)

In [ ]:
# All results
means = NN_mod_tuned.cv_results_['mean_test_score']
stds = NN_mod_tuned.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, NN_mod_tuned.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
preds_NN_train = NN_mod_tuned.predict(NN_train)
preds_NN_test = NN_mod_tuned.predict(NN_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_NN_train), results_dir + 'preds.NN.train_agg.feather')
feather.write_feather(pd.DataFrame(preds_NN_test), results_dir + 'preds.NN.test_agg.feather')

In [ ]:
print("RMSE of the training set:", np.sqrt(mean_squared_error(Y_train, preds_NN_train)))
print("R^2 of the training set:", r2_score(Y_train, preds_NN_train))

In [ ]:
print("RMSE of the test set:", np.sqrt(mean_squared_error(Y_test, preds_NN_test)))
print("R^2 of the test set:", r2_score(Y_test, preds_NN_test))